### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import warnings

#Disable warnings
warnings.filterwarnings("ignore")

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
# Combine the datasets into a single dataframe object
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


# Get the number of schools and store it in a dataframe, rename the column name to "Total Schools"
number_of_schools = len(school_data_complete["school_name"].unique())
summary_df = pd.DataFrame({number_of_schools})
summary_df = summary_df.rename(columns={0:"Total Schools"})

# Get the number of students and add it as a new column"Total Students" to the existing "Summary" dataframe
number_of_students = school_data_complete["Student ID"].count()
summary_df["Total Students"] = number_of_students
summary_df["Total Students"] = summary_df["Total Students"].map("{:,}".format)

# Get the total budget and add it as a new column "Total Budget" to the existing "Summary" dataframe
total_budget = school_data["budget"].sum(axis = 0, skipna = True)
summary_df["Total Budget"] = total_budget
summary_df["Total Budget"] = summary_df["Total Budget"].map("${:,.2f}".format)

#Use the dataframe's mean() function to calculate the average values for math, reading and overall scores.
# Update Summary dataframe to include these new averages
avg_math_score = school_data_complete["math_score"].mean()
summary_df["Avg Math Score"] = avg_math_score

avg_reading_score = school_data_complete["reading_score"].mean()
summary_df["Avg Reading Score"] = avg_reading_score

math_pass_percent = (len(school_data_complete.loc[(school_data_complete["math_score"] >= 70),"math_score"])/number_of_students)*100
summary_df["% passing Math"] = math_pass_percent

reading_pass_percent = (len(school_data_complete.loc[(school_data_complete["reading_score"] >= 70),"reading_score"])/number_of_students)*100
summary_df["% passing Reading"] = reading_pass_percent

overall_pass_percent = (len(school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70),"reading_score"])/number_of_students)*100
summary_df["% passing Overall"] = overall_pass_percent

summary_df


,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% passing Math,% passing Reading,% passing Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Createa new dataframe to store "School name", "Type", "Size" and "Budget" columns from school_data
#Set index for this new dataframe and rename the column names to more meaningful names
school_summary_df = school_data[["school_name","type","size","budget"]]
school_summary_df = school_summary_df.set_index("school_name")
school_summary_df = school_summary_df.rename(columns={"type":"School Type","size":"Total Students","budget":"Total School Budget"})

#Calculate Per Student Budget and store it in a new column.
#Format Budget columns to float with upto 2 digits after decimal
school_summary_df["Per Student Budget"] = school_summary_df["Total School Budget"]/school_summary_df["Total Students"]
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:.2f}".format)

#Create a new dataframe to store School Name, Math and Reading scores and size columns from combined master dataset(school_data_complete), contains data from both school_data.csv and student_data.csv
school_data_raw = school_data_complete[["school_name","reading_score","math_score","size"]]

#Grouping data by School Name
grouped_school_data = school_data_raw.groupby(['school_name'])

#Calculate means for Math and Reading scores, combine these datasets into a single dataframe on School Name
reading_ds = grouped_school_data["reading_score"].mean()
math_ds = grouped_school_data["math_score"].mean()
mean_data_df = pd.DataFrame({"school_name":reading_ds.index,"Average Math Score":math_ds.values,"Average Reading Score": reading_ds.values})
combined_school_df = pd.merge(school_summary_df,mean_data_df,on="school_name")
combined_school_df = combined_school_df.sort_values("school_name")


#Get unique school names, iterate through this list to calculate school wise passing perctages in Math, reading and Overall
distinct_schools = school_data_complete["school_name"].unique()
distinct_schools

math_list = []
schools_list = []
reading_list = []
overall_list = []
for school in distinct_schools:
    math_pass_students = len(school_data_complete.loc[((school_data_complete["math_score"] >= 70) & (school_data_complete["school_name"] == school)),"math_score"])
    reading_pass_students = len(school_data_complete.loc[((school_data_complete["reading_score"] >= 70) & (school_data_complete["school_name"] == school)),"reading_score"])
    overall_pass_students = len(school_data_complete.loc[((school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70) & (school_data_complete["school_name"] == school)),"math_score"])
    school_students = len(school_data_complete.loc[(school_data_complete["school_name"] == school),"math_score"])
    math_pass_percent = (math_pass_students/school_students)*100
    reading_pass_percent = (reading_pass_students/school_students)*100
    overall_pass_percent = (overall_pass_students/school_students)*100
    schools_list.append(school)
    math_list.append(math_pass_percent)
    reading_list.append(reading_pass_percent)
    overall_list.append(overall_pass_percent)
    
school_data = {"school_name" : schools_list,
                "Percent Math Pass" : math_list,
               "Percent Reading Pass" : reading_list,
               "Percent Overall Pass" : overall_list}

school_pass_df = pd.DataFrame(school_data)

combined_school_df = pd.merge(combined_school_df,school_pass_df,on="school_name")
            
new_combined_school_df = combined_school_df.set_index("school_name")
new_combined_school_df
               

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Math Pass,Percent Reading Pass,Percent Overall Pass
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#Sort the dataset in descending order and pick top 5 rows
top_performers_df = combined_school_df.sort_values("Percent Overall Pass", ascending = False)
new_top_performers_df= top_performers_df.set_index("school_name")
new_top_performers_df.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Math Pass,Percent Reading Pass,Percent Overall Pass
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
#Sort the dataset in ascending order and pick top 5 rows
bottom_performers_df = combined_school_df.sort_values("Percent Overall Pass")
new_bottom_performers_df = bottom_performers_df.set_index("school_name")
new_bottom_performers_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Math Pass,Percent Reading Pass,Percent Overall Pass
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
#Create different dataframes using "loc" to filter data based on grade
#For each dataframe, calculate means for Math and Reading scores
ninth_grade_df = school_data_complete.loc[school_data_complete["grade"] == "9th",:]
ninth_grade_df = ninth_grade_df.groupby("school_name")
ninth_avg_math = ninth_grade_df["math_score"].mean()
ninth_avg_reading = ninth_grade_df["reading_score"].mean()

tenth_grade_df = school_data_complete.loc[school_data_complete["grade"] == "10th",:]
tenth_grade_df = tenth_grade_df.groupby("school_name")
tenth_avg_math = tenth_grade_df["math_score"].mean()
tenth_avg_reading = tenth_grade_df["reading_score"].mean()

eleventh_grade_df = school_data_complete.loc[school_data_complete["grade"] == "11th",:]
eleventh_grade_df = eleventh_grade_df.groupby("school_name")
eleventh_avg_math = eleventh_grade_df["math_score"].mean()
eleventh_avg_reading = eleventh_grade_df["reading_score"].mean()

twelfth_grade_df = school_data_complete.loc[school_data_complete["grade"] == "12th",:]
twelfth_grade_df = twelfth_grade_df.groupby("school_name")
twelfth_avg_math = twelfth_grade_df["math_score"].mean()
twelfth_avg_reading = twelfth_grade_df["reading_score"].mean()

#Create a final dataframe by adding math scores for all grades
students_math_report = pd.DataFrame({"SCHOOL NAME": ninth_avg_math.index,"9th":ninth_avg_math.values,"10th":tenth_avg_math.values,"11th":eleventh_avg_math.values,"12th": twelfth_avg_math.values})
students_math_report.set_index("SCHOOL NAME", inplace = True)
students_math_report

,9th,10th,11th,12th
SCHOOL NAME,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
#Create similar dataframe as above by adding columns related to reading scores for all grades
students_reading_report = pd.DataFrame({"SCHOOL NAME": ninth_avg_math.index,"9th":ninth_avg_reading.values,"10th":tenth_avg_reading.values,"11th":eleventh_avg_reading.values,"12th": twelfth_avg_reading.values})
students_reading_report.set_index("SCHOOL NAME", inplace = True)
students_reading_report

,9th,10th,11th,12th
SCHOOL NAME,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# Use bins to split data based on Per Student Budget columns

bins = [0,584,629,644,675]

combined_school_df["Per Student Budget"] = [x.strip("$") for x in combined_school_df["Per Student Budget"]]
combined_school_df["Per Student Budget"]  = combined_school_df["Per Student Budget"].astype(float)
group_names = ["<$585","\$585-$629","\$630-$644","\$645-$675"]
combined_school_df["School Spending"] = pd.cut(combined_school_df["Per Student Budget"], bins, labels = group_names, include_lowest= True)
school_report = combined_school_df[["school_name","Average Math Score", "Average Reading Score","Percent Math Pass","Percent Reading Pass","Percent Overall Pass", "School Spending"]]
school_report["Average Math Score"] = pd.to_numeric(school_report["Average Math Score"])
#Groupby using "School Spending". Use mean() function to display averages
school_report = school_report.groupby("School Spending")
new_school_report = school_report.mean()
new_school_report["Average Math Score"] = new_school_report["Average Math Score"].map("{:.2f}".format)
new_school_report["Average Reading Score"] = new_school_report["Average Reading Score"].map("{:.2f}".format)
new_school_report["Percent Math Pass"] = new_school_report["Percent Math Pass"].map("{:.2f}".format)
new_school_report["Percent Reading Pass"] = new_school_report["Percent Reading Pass"].map("{:.2f}".format)
new_school_report["Percent Overall Pass"] = new_school_report["Percent Overall Pass"].map("{:.2f}".format)
new_school_report

,Average Math Score,Average Reading Score,Percent Math Pass,Percent Reading Pass,Percent Overall Pass
School Spending,,,,,
<$585,83.46,83.93,93.46,96.61,90.37
\$585-$629,81.90,83.16,87.13,92.72,81.42
\$630-$644,78.52,81.62,73.48,84.39,62.86
\$645-$675,77.00,81.03,66.16,81.13,53.53


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

In [8]:
#Use bins to split data based on school size
bins_schoolsize = [0,999,1999,4999]
schoolsize_name = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]
combined_school_df["School Size"] = pd.cut(combined_school_df["Total Students"], bins_schoolsize, labels = schoolsize_name, include_lowest= True)
school_report_size = combined_school_df[["school_name","Average Math Score", "Average Reading Score","Percent Math Pass", "Percent Reading Pass","Percent Overall Pass","School Size"]]
#Groupby using school size. Use mean() to display averages
school_report_size = school_report_size.groupby("School Size")
new_school_report_size = school_report_size.mean()
new_school_report_size["Average Math Score"] = new_school_report_size["Average Math Score"].map("{:.2f}".format)
new_school_report_size["Average Reading Score"] = new_school_report_size["Average Reading Score"].map("{:.2f}".format)
new_school_report_size["Percent Math Pass"] = new_school_report_size["Percent Math Pass"].map("{:.2f}".format)
new_school_report_size["Percent Reading Pass"] = new_school_report_size["Percent Reading Pass"].map("{:.2f}".format)
new_school_report_size["Percent Overall Pass"] = new_school_report_size["Percent Overall Pass"].map("{:.2f}".format)
new_school_report_size

,Average Math Score,Average Reading Score,Percent Math Pass,Percent Reading Pass,Percent Overall Pass
School Size,,,,,
Small(<1000),83.82,83.93,93.55,96.10,89.88
Medium(1000-2000),83.37,83.86,93.60,96.79,90.62
Large(2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [9]:
#Group by "School Type", use mean() to display averages
school_type_report = combined_school_df[["school_name","Average Math Score","Average Reading Score","Percent Math Pass" ,"Percent Reading Pass","Percent Overall Pass","School Type"]]
school_type_report = school_type_report.groupby("School Type")
new_school_type_report = school_type_report.mean()
new_school_type_report["Average Math Score"] = new_school_type_report["Average Math Score"].map("{:.2f}".format)
new_school_type_report["Average Reading Score"] = new_school_type_report["Average Reading Score"].map("{:.2f}".format)
new_school_type_report["Percent Math Pass"] = new_school_type_report["Percent Math Pass"].map("{:.2f}".format)
new_school_type_report["Percent Reading Pass"] = new_school_type_report["Percent Reading Pass"].map("{:.2f}".format)
new_school_type_report["Percent Overall Pass"] = new_school_type_report["Percent Overall Pass"].map("{:.2f}".format)
new_school_type_report

,Average Math Score,Average Reading Score,Percent Math Pass,Percent Reading Pass,Percent Overall Pass
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
